In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Advanced SQL


As in the `IW_SQL` notebook, we'll be looking at the Structured Query Language (SQL).  Here we'll be extending on what we covered, particularly by looking at table manipulation, views, and sessions v transactions (and locking).

Make sure you've created the tables and run the code in `IW_SQL` first!  We'll be re-using the same tables a bit, so we'll be drawing from the same testdb.sqlite file

In [ ]:
import pandas as pd

In [ ]:
%load_ext sql
%sql sqlite:///testdb.sqlite

In [ ]:
%%sql

--#This is sqlite specific, it's different for every flavor of SQL
SELECT * FROM sqlite_master WHERE type='table';

At this point, it would be good to review the `IW_SQL` notebook.  Particularly, JOINs, UNIONs, and subqueries, as they are common points of confusion.


## Sessions and transactions


When you connect to an SQL database, you create a _session_.  You can think of this just like an ssh session - you log in/connect, you can change things around, run commands, and as long as you don't save them they'll be lost when you log out/disconnect.  Also, usually any admin changes made to the database (changing table permissions, etc) won't affect your session until you disconnect and reconnect.

A session consists of a number of _transactions_.  These are atomic blocks of SQL commands.  These are equivalent to our cells in the jupyter notebook, in that they're run together as a unit.  However, in most SQL setups, they actually extend over multiple commands you send to the database until you either `COMMIT` or `ROLLBACK`.  COMMIT saves all your changes and makes them "permanent" in the database, while ROLLBACK resets the database state to what it was before your transaction block.  In this sense, every transaction maintains a state in the database, so changes that someone else makes *while* a transaction is running won't be visible.  We don't need them here in Jupyter since it defaults to autocommit - every cell is treated as a single transaction with an implied `COMMIT` at the end.

One might imagine any number of scenarios where this could be a problem - two people trying to alter the same table, for example.  The DB management software is pretty smart and serializes changes in such a way that they make sense, but you can occasionally run in to problems and a table can _lock_ - the session will freeze while it waits for conflicting changes to resolve.  Very similar to a race condition or a mutex lock.  Generally at that point the DB Admin has to fix it, or you can try to ROLLBACK your changes.

Easy to serialize:
- SELECT
- INSERT

Hard to serialize, can lead to conflicts:
- UPDATE
- DELETE
- ALTER TABLE, DROP TABLE, etc.


## Manipulating tables


Let's look at manipulating tables - we'll look at this for temporary tables, as this is most likely what you'll be doing as someone analyzing data, but to do it for a real table is identical (save for the changes in the CREATE statement).

In [ ]:
%%sql

DROP TABLE IF EXISTS jfriends;

/*#Create a table as a selection from another one.  
#Of course, selection can be arbitrarily complex*/

CREATE TEMP TABLE jfriends AS 
SELECT * FROM friends WHERE name like 'J%';

SELECT * FROM jfriends;

In [ ]:
%%sql

--#Note that jfriends isn't in the list, it hasn't been persisted.  It will vanish
SELECT * FROM sqlite_master WHERE type='table';

Question: Why make a temporary table?  What's the strength and weakness?

In [ ]:
%%sql

DROP TABLE IF EXISTS new_friends;

CREATE TEMP TABLE new_friends (
    name              TEXT NOT NULL,
    city              TEXT NOT NULL,
    favorite_color    TEXT
);

INSERT INTO new_friends --(name, city, favorite_color) <- this part is optional
SELECT name, city, favorite_color FROM friends WHERE name like 'J%';

SELECT * FROM new_friends;

In [ ]:
%%sql

--#Another option to create a blank table, if we want to keep the structure of an old one
DROP TABLE IF EXISTS blank_friends;

CREATE TEMP TABLE blank_friends AS
SELECT * FROM friends WHERE 1=0;

SELECT * FROM blank_friends;

In [ ]:
%%sql

--#This is a bit SQL flavor dependent, check your documentation

PRAGMA table_info(blank_friends)

In [ ]:
%%sql

--#Let's get rid of the J names, and put some new friends in the table

--# This works in some SQL flavors, but not sqlite: TRUNCATE TABLE new_friends;

--#You can of course put any WHERE statement there, but this one wipes the whole table
--#Technically, the DELETE FROM new_friends; does the same thing
DELETE FROM new_friends WHERE 1 = 1;

INSERT INTO new_friends VALUES ('Mary', 'Austin', NULL), ('Brian', 'Houston', NULL);

SELECT * FROM new_friends;

In [ ]:
%%sql

--#Let's say Mary decided on a favorite color.  Let's update the table correspondingly
UPDATE new_friends SET favorite_color = 'Blue' WHERE name = 'Mary';

SELECT * FROM new_friends;

## Views


A view is a saved SQL query, that you can treat as a table.  It's "live", in the sense that it reflects changes to the underlying table.  They can be permanent or temporary, just like a table.

In [ ]:
%%sql

CREATE VIEW jfriends_view AS SELECT * FROM friends WHERE name like 'J%';

In [ ]:
%%sql

--#same content as jfriends

SELECT * FROM jfriends_view;

In [ ]:
%%sql

INSERT INTO friends (name, city, favorite_color) VALUES ('Joe', 'New York', 'Red');

In [ ]:
%%sql

SELECT * FROM jfriends_view;

In [ ]:
%%sql

SELECT * FROM jfriends;

Question: What is the benefit and drawback of a view?  When might you want to have a permanent view in the database?

## Advanced query commands


In IW_SQL, we neglected to cover a few useful topics.  They may have been mentioned in class, but are not written down.  They are:

- DISTINCT, COUNT(DISTINCT ...)
- HAVING
- WITH
- Dealing with NULLs
- UDFs (these are very SQL flavor specific, but should be mentioned)

In [ ]:
%%sql

--#DISTINCT does exactly what you think it does
SELECT DISTINCT name FROM friends;

In [ ]:
%%sql

--#COUNT(DISTINCT col_name) can be very useful
SELECT COUNT(DISTINCT city) AS num_cities FROM friends;

In [ ]:
%%sql

--#Having is just the GROUP BY equivalent of WHERE.  Note it doesn't respect column aliases
SELECT city, count(name) as friends FROM friends GROUP BY city HAVING count(name) > 1;

`WITH` is another way of handling subqueries.  It only allows relatively simple ones, but makes everything much cleaner

In [ ]:
%%sql

--#Let's add some more sweater prices

INSERT INTO merchandise VALUES ('red', 39.99), ('blue', 59.99);

select * from merchandise

In [ ]:
%%sql

select * from friends

In [ ]:
%%sql

--#We'll want to get the cheapest variety of each sweater color
--#Note that this only returns people that have a sweater color available

WITH cheapest AS (SELECT sweater_color, MIN(price) as price FROM merchandise GROUP BY 1),
lower AS (SELECT name, city, LOWER(favorite_color) as color FROM friends)
SELECT name, city, color, price FROM lower JOIN cheapest ON sweater_color = color;

So, in the above, we didn't do anything for people who didn't have a favorite color.  Let's instead give them a default -- the cheapest one available.  We'll use CASE WHEN, which all flavors of SQL have, but many also have COALESCE and similar for dealing with this case.

In [ ]:
%%sql

WITH cheapest_color AS (SELECT sweater_color FROM merchandise ORDER BY price LIMIT 1),
cheapest AS (SELECT sweater_color, MIN(price) as price FROM merchandise GROUP BY 1),
lower AS (SELECT name, city, 
          CASE WHEN favorite_color IN (SELECT sweater_color FROM merchandise) 
          THEN LOWER(favorite_color)
          ELSE (SELECT sweater_color FROM cheapest_color) END AS color FROM friends)
SELECT name, city, color, price FROM lower JOIN cheapest ON sweater_color = color;

Question: What's the drawback to using WITH instead of a subquery?

*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*